In [ ]:
### Establish Working Directory

import pyodbc
import pandas as pd

connection_string = (
    "DRIVER={ODBC Driver 18 for SQL Server};"
    "SERVER=localhost;"
    "DATABASE=EngagementMiser;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"
)

query = """
SELECT top 100 tweet_id, text
FROM dbo.Tweets_Sample_4M
WHERE text IS NOT NULL AND LEN(text) > 10;
"""

with pyodbc.connect(connection_string) as conn:
    df = pd.read_sql(query, conn)

print(f"Loaded {len(df)} tweets")


In [ ]:
"""
Train a text-only transformer with soft labels pulled from SQL Server,
then score any tweet_id on demand (0–1 probability-like score).

Table used:
  [EngagementMiser].[dbo].[Hyperbole_Falsehood_tweets_annot]
Columns (as provided):
  tweet_id (str/int), text (str), author_id, source, label_confidence (float in [0,1])

Notes:
- We treat label_confidence as a *soft* target and optimize BCEWithLogitsLoss.
- Model outputs a single logit; we apply sigmoid at inference to get [0,1].
- Only tweet `text` is used as input features.
"""

import os
import math
import random
import numpy as np
import pandas as pd
from dataclasses import dataclass

# Torch & HF
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup,
)

# Progress bar
from tqdm.auto import tqdm

# --- SQL connectivity (via SQLAlchemy + pyodbc) ---
# Fill these in for your environment. Prefer env vars where possible.
SQL_SERVER   = os.getenv("SQL_SERVER",   "<YOUR-SERVER-HOST>")   # e.g., "localhost" or "SQLHOST\\SQLEXPRESS"
SQL_DB       = os.getenv("SQL_DB",       "EngagementMiser")
SQL_UID      = os.getenv("SQL_UID",      "<USERNAME>")
SQL_PWD      = os.getenv("SQL_PWD",      "<PASSWORD>")
SQL_DRIVER   = os.getenv("SQL_DRIVER",   "ODBC Driver 17 for SQL Server")  # or "ODBC Driver 18 for SQL Server"



from sqlalchemy import create_engine, text as sql_text
# Connection string format for SQL Server over ODBC
CONN_STR = (
    f"mssql+pyodbc://{SQL_UID}:{SQL_PWD}@{SQL_SERVER}/{SQL_DB}"
    f"?driver={SQL_DRIVER.replace(' ', '+')}"
)
engine = create_engine(CONN_STR, fast_executemany=True)

# ----------------------
# Reproducibility helpers
# ----------------------
def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# ----------------------
# Config (tweak as needed)
# ----------------------
MODEL_NAME = "roberta-base"     # solid default; swap for "bert-base-uncased", etc.
MAX_LEN    = 160                # typical tweet length cap (after URLs/handles)
BATCH_SIZE = 16
EPOCHS     = 3
LR         = 2e-5               # standard fine-tune LR
WARMUP_PCT = 0.1

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)

# ----------------------
# 1) Load from SQL Server
# ----------------------
QUERY = """
SELECT
    CAST(tweet_id AS VARCHAR(32)) AS tweet_id,   -- keep id as string to be safe
    text,
    CAST(label_confidence AS FLOAT) AS label_confidence
FROM [EngagementMiser].[dbo].[Hyperbole_Falsehood_tweets_annot]
WHERE text IS NOT NULL
  AND label_confidence IS NOT NULL
"""

df = pd.read_sql_query(sql_text(QUERY), engine)

# Basic sanity checks / cleaning
df["text"] = df["text"].astype(str).str.strip()
df = df[(df["text"].str.len() > 0)]
# Ensure labels are in [0,1]; if your table guarantees this, the clip is harmless:
df["label_confidence"] = df["label_confidence"].astype(float).clip(0.0, 1.0)

print(f"Loaded {len(df):,} rows from SQL.")

# ----------------------
# 2) Train / Val split
# ----------------------
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(
    df, test_size=0.1, random_state=42, stratify=None  # regression/soft labels -> no stratify
)
print(f"Train: {len(train_df):,} | Val: {len(val_df):,}")

# ----------------------
# 3) Tokenizer & Dataset
# ----------------------
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)

class TweetDataset(Dataset):
    """Wraps text + soft label for the dataloader."""
    def __init__(self, texts, labels, tokenizer, max_len=160):
        self.texts = list(texts)
        self.labels = list(labels)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        t = self.texts[idx]
        y = float(self.labels[idx])  # soft label in [0,1]

        enc = self.tokenizer(
            t,
            padding="max_length",
            truncation=True,
            max_length=self.max_len,
            return_tensors="pt",
        )

        # Flatten batch dimension (since return_tensors='pt' adds it)
        item = {
            "input_ids":      enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            # No token_type_ids for RoBERTa; for BERT you could include it if present
            "labels":         torch.tensor(y, dtype=torch.float),
        }
        return item

train_ds = TweetDataset(train_df["text"], train_df["label_confidence"], tokenizer, MAX_LEN)
val_ds   = TweetDataset(val_df["text"],   val_df["label_confidence"],   tokenizer, MAX_LEN)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, pin_memory=True)

# ----------------------
# 4) Model (single-logit head)
# ----------------------
# We’ll still use AutoModelForSequenceClassification for convenience,
# but we’ll compute BCEWithLogitsLoss *manually* so it behaves like a soft-label classifier.
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=1  # single regression-style logit
)
model.to(DEVICE)

# Soft-label objective: binary cross-entropy with logits
criterion = nn.BCEWithLogitsLoss()

# Optimizer / Scheduler
optimizer = AdamW(model.parameters(), lr=LR)
num_training_steps = EPOCHS * len(train_loader)
num_warmup_steps = int(WARMUP_PCT * num_training_steps)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_warmup_steps, num_training_steps=num_training_steps
)

# Optional: grad clipping value
MAX_GRAD_NORM = 1.0

# ----------------------
# 5) Training / Evaluation loops
# ----------------------
def train_one_epoch(epoch: int):
    model.train()
    total_loss = 0.0

    pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [train]")
    for batch in pbar:
        optimizer.zero_grad(set_to_none=True)

        input_ids      = batch["input_ids"].to(DEVICE, non_blocking=True)
        attention_mask = batch["attention_mask"].to(DEVICE, non_blocking=True)
        labels         = batch["labels"].to(DEVICE, non_blocking=True)

        # Forward: we don't pass labels to the model so we can control the loss function
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits  = outputs.logits.squeeze(-1)  # shape: (B,)

        loss = criterion(logits, labels)  # soft labels in [0,1]
        loss.backward()

        # Gradient clipping for stability
        torch.nn.utils.clip_grad_norm_(model.parameters(), MAX_GRAD_NORM)

        optimizer.step()
        scheduler.step()

        total_loss += loss.item()
        pbar.set_postfix({"loss": f"{loss.item():.4f}"})

    return total_loss / max(1, len(train_loader))

@torch.no_grad()
def evaluate():
    model.eval()
    total_loss = 0.0
    all_preds, all_targets = [], []

    pbar = tqdm(val_loader, desc="Validation")
    for batch in pbar:
        input_ids      = batch["input_ids"].to(DEVICE, non_blocking=True)
        attention_mask = batch["attention_mask"].to(DEVICE, non_blocking=True)
        labels         = batch["labels"].to(DEVICE, non_blocking=True)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits  = outputs.logits.squeeze(-1)

        loss = criterion(logits, labels)
        total_loss += loss.item()

        # Convert logits -> probabilities with sigmoid for reporting
        probs = torch.sigmoid(logits)
        all_preds.append(probs.cpu().numpy())
        all_targets.append(labels.cpu().numpy())

    all_preds   = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    # Report a few friendly metrics for soft labels
    mae = float(np.mean(np.abs(all_preds - all_targets)))
    mse = float(np.mean((all_preds - all_targets) ** 2))

    return total_loss / max(1, len(val_loader)), mae, mse

best_val = math.inf
for epoch in range(EPOCHS):
    train_loss = train_one_epoch(epoch)
    val_loss, val_mae, val_mse = evaluate()
    print(
        f"Epoch {epoch+1}: train_loss={train_loss:.4f} | "
        f"val_loss={val_loss:.4f} | val_MAE={val_mae:.4f} | val_MSE={val_mse:.4f}"
    )

    # (Optional) Simple checkpointing on validation loss
    if val_loss < best_val:
        best_val = val_loss
        os.makedirs("checkpoints", exist_ok=True)
        model.save_pretrained("checkpoints/text_softlabel_roberta")
        tokenizer.save_pretrained("checkpoints/text_softlabel_roberta")
        print("✅ Saved checkpoint -> checkpoints/text_softlabel_roberta")

# ----------------------
# 6) Single-text inference helper
# ----------------------
@torch.no_grad()
def predict_single_text(text: str) -> float:
    """
    Returns a float in [0,1] (sigmoid of the single logit).
    """
    model.eval()
    enc = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=MAX_LEN,
        return_tensors="pt",
    )
    input_ids      = enc["input_ids"].to(DEVICE)
    attention_mask = enc["attention_mask"].to(DEVICE)

    logits = model(input_ids=input_ids, attention_mask=attention_mask).logits
    # shape: (1, 1) -> squeeze to scalar
    score = torch.sigmoid(logits.squeeze()).item()
    return float(score)

# ----------------------
# 7) Score a tweet by tweet_id (pulled from SQL)
# ----------------------
def score_tweet_id(tweet_id: str | int) -> float:
    """
    Look up the tweet text by tweet_id in the SQL table and return a [0,1] score.
    Raises ValueError if tweet_id not found.
    """
    query = sql_text(
        """
        SELECT TOP 1 text
        FROM [EngagementMiser].[dbo].[Hyperbole_Falsehood_tweets_annot]
        WHERE CAST(tweet_id AS VARCHAR(32)) = :tid
        """
    )
    with engine.connect() as conn:
        row = conn.execute(query, {"tid": str(tweet_id)}).fetchone()
    if not row or not row[0]:
        raise ValueError(f"tweet_id {tweet_id} not found or has no text.")
    return predict_single_text(str(row[0]))

# ----------------------
# 8) Example usage
# ----------------------
if __name__ == "__main__":
    # Example: score one of your provided ids
    try:
        example_id = "1489594096360550000"
        score = score_tweet_id(example_id)
        print(f"Score for tweet_id {example_id}: {score:.4f} (0=not hyperbolic/false, 1=very)")
    except Exception as e:
        print("Scoring error:", e)
